In [8]:
import os

from sklearn.feature_extraction.text import TfidfVectorizer

from knodle.model import LogisticRegressionModel
from joblib import load, dump
import pandas as pd
from torch import Tensor

from knodle.trainer import KnnDenoising
import logging
import torch

# Imports
from knodle.trainer.baseline.baseline import SimpleDsModelTrainer
from knodle.model import LogisticRegressionModel
from knodle.trainer.config.TrainerConfig import TrainerConfig
from snorkel.classification import cross_entropy_with_probs
from torch.optim import SGD

logger = logging.getLogger(__name__)

OUTPUT_CLASSES = 2

os.chdir('/Users/sandro/repo/knodle/')


In [22]:
# Helper functions

def read_evaluation_data():
    imdb_dataset = pd.read_csv("tutorials/ImdbDataset/imdb_data_preprocessed.csv")
    applied_lfs = load("tutorials/ImdbDataset/applied_lfs.lib")
    return imdb_dataset, applied_lfs


def create_tfidf_values(text_data: [str]):
    if os.path.exists("tutorials/ImdbDataset/tfidf.lib"):
        cached_data = load("tutorials/ImdbDataset/tfidf.lib")
        if cached_data.shape == text_data.shape:
            return cached_data
    logger.info("Create new")

    vectorizer = TfidfVectorizer()
    transformed_data = vectorizer.fit_transform(text_data)
    dump(transformed_data, "tutorials/ImdbDataset/tfidf.lib")
    return transformed_data


def binary_accuracy(y_pred, y_true):
    """Function to calculate binary accuracy per batch"""
    y_true = y_true if len(y_true.shape) == 1 else torch.argmax(y_true, dim=-1)
    y_pred_max = torch.argmax(y_pred, dim=-1)
    correct_pred = (y_pred_max == y_true).float()
    acc = correct_pred.sum() / len(correct_pred)
    return acc

def test(model, test_features, test_labels):
    model.eval()

    predictions = model(test_features)

    acc = binary_accuracy(predictions, test_labels)
    print(acc.numpy())
    

# Evaluation example


In [23]:
imdb_dataset, rule_matches = read_evaluation_data()
imdb_sample = imdb_dataset.sample(1000, random_state=123)
rule_matches_sample = rule_matches[imdb_sample.index]
assert len(imdb_sample) == len(rule_matches_sample)

tfidf_values = create_tfidf_values(imdb_sample.reviews_preprocessed.values)

tfidf_tensor = Tensor(tfidf_values.toarray())


train_tfidf = tfidf_values[:800]
train_tensor = Tensor(train_tfidf.toarray())
train_rule_matches = rule_matches_sample[:800]

test_tfidf = tfidf_values[801:]
test_tensor = Tensor(test_tfidf.toarray())


test_label_tensor = Tensor(imdb_sample[801:].label_id.values)

2020-11-23 15:58:23,267 __main__     INFO     Create new


## Baseline Model

In [24]:
model = LogisticRegressionModel(train_tensor.shape[1], OUTPUT_CLASSES)
trainer_config = TrainerConfig(model, criterion=cross_entropy_with_probs, optimizer_=SGD(model.parameters(), lr=5))

trainer = SimpleDsModelTrainer(model, trainer_config=trainer_config)
trainer.train(inputs=train_tensor, rule_matches=train_rule_matches, epochs=1000)

2020-11-23 15:58:24,893 knodle.trainer.ds_model_trainer.ds_model_trainer INFO     Initalized trainer with custom model config: {'criterion': <function cross_entropy_with_probs at 0x13dfd51e0>, 'batch_size': 32, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 5
    momentum: 0
    nesterov: False
    weight_decay: 0
), 'output_classes': 2}
2020-11-23 15:58:24,918 knodle.trainer.baseline.baseline INFO     ======================================
2020-11-23 15:58:24,919 knodle.trainer.baseline.baseline INFO     Training starts
2020-11-23 15:58:24,919 knodle.trainer.baseline.baseline INFO     ======================================
2020-11-23 15:58:24,919 knodle.trainer.baseline.baseline INFO     Epoch: 0
2020-11-23 15:58:24,929 knodle.trainer.baseline.baseline INFO     Loss is: 0.6928479075431824
2020-11-23 15:58:24,938 knodle.trainer.baseline.baseline INFO     Epoch: 1
2020-11-23 15:58:24,946 knodle.trainer.baseline.baseline INFO     Loss is: 0.6095340251922607
2020-11-23 15:5

2020-11-23 15:58:25,716 knodle.trainer.baseline.baseline INFO     Loss is: 0.3778458535671234
2020-11-23 15:58:25,725 knodle.trainer.baseline.baseline INFO     Epoch: 45
2020-11-23 15:58:25,734 knodle.trainer.baseline.baseline INFO     Loss is: 0.3763420879840851
2020-11-23 15:58:25,743 knodle.trainer.baseline.baseline INFO     Epoch: 46
2020-11-23 15:58:25,751 knodle.trainer.baseline.baseline INFO     Loss is: 0.37484991550445557
2020-11-23 15:58:25,760 knodle.trainer.baseline.baseline INFO     Epoch: 47
2020-11-23 15:58:25,769 knodle.trainer.baseline.baseline INFO     Loss is: 0.3733694553375244
2020-11-23 15:58:25,778 knodle.trainer.baseline.baseline INFO     Epoch: 48
2020-11-23 15:58:25,787 knodle.trainer.baseline.baseline INFO     Loss is: 0.37190043926239014
2020-11-23 15:58:25,797 knodle.trainer.baseline.baseline INFO     Epoch: 49
2020-11-23 15:58:25,805 knodle.trainer.baseline.baseline INFO     Loss is: 0.37044283747673035
2020-11-23 15:58:25,814 knodle.trainer.baseline.basel

2020-11-23 15:58:26,572 knodle.trainer.baseline.baseline INFO     Epoch: 93
2020-11-23 15:58:26,580 knodle.trainer.baseline.baseline INFO     Loss is: 0.3163783848285675
2020-11-23 15:58:26,590 knodle.trainer.baseline.baseline INFO     Epoch: 94
2020-11-23 15:58:26,598 knodle.trainer.baseline.baseline INFO     Loss is: 0.31535065174102783
2020-11-23 15:58:26,607 knodle.trainer.baseline.baseline INFO     Epoch: 95
2020-11-23 15:58:26,616 knodle.trainer.baseline.baseline INFO     Loss is: 0.31433069705963135
2020-11-23 15:58:26,625 knodle.trainer.baseline.baseline INFO     Epoch: 96
2020-11-23 15:58:26,634 knodle.trainer.baseline.baseline INFO     Loss is: 0.31331849098205566
2020-11-23 15:58:26,643 knodle.trainer.baseline.baseline INFO     Epoch: 97
2020-11-23 15:58:26,651 knodle.trainer.baseline.baseline INFO     Loss is: 0.31231388449668884
2020-11-23 15:58:26,660 knodle.trainer.baseline.baseline INFO     Epoch: 98
2020-11-23 15:58:26,669 knodle.trainer.baseline.baseline INFO     Loss

2020-11-23 15:58:27,425 knodle.trainer.baseline.baseline INFO     Epoch: 141
2020-11-23 15:58:27,433 knodle.trainer.baseline.baseline INFO     Loss is: 0.2746802568435669
2020-11-23 15:58:27,442 knodle.trainer.baseline.baseline INFO     Epoch: 142
2020-11-23 15:58:27,451 knodle.trainer.baseline.baseline INFO     Loss is: 0.2739544212818146
2020-11-23 15:58:27,460 knodle.trainer.baseline.baseline INFO     Epoch: 143
2020-11-23 15:58:27,468 knodle.trainer.baseline.baseline INFO     Loss is: 0.2732335031032562
2020-11-23 15:58:27,477 knodle.trainer.baseline.baseline INFO     Epoch: 144
2020-11-23 15:58:27,485 knodle.trainer.baseline.baseline INFO     Loss is: 0.27251750230789185
2020-11-23 15:58:27,495 knodle.trainer.baseline.baseline INFO     Epoch: 145
2020-11-23 15:58:27,504 knodle.trainer.baseline.baseline INFO     Loss is: 0.27180635929107666
2020-11-23 15:58:27,513 knodle.trainer.baseline.baseline INFO     Epoch: 146
2020-11-23 15:58:27,521 knodle.trainer.baseline.baseline INFO     

2020-11-23 15:58:28,266 knodle.trainer.baseline.baseline INFO     Epoch: 189
2020-11-23 15:58:28,274 knodle.trainer.baseline.baseline INFO     Loss is: 0.2447337508201599
2020-11-23 15:58:28,284 knodle.trainer.baseline.baseline INFO     Epoch: 190
2020-11-23 15:58:28,292 knodle.trainer.baseline.baseline INFO     Loss is: 0.24420204758644104
2020-11-23 15:58:28,302 knodle.trainer.baseline.baseline INFO     Epoch: 191
2020-11-23 15:58:28,310 knodle.trainer.baseline.baseline INFO     Loss is: 0.24367362260818481
2020-11-23 15:58:28,319 knodle.trainer.baseline.baseline INFO     Epoch: 192
2020-11-23 15:58:28,327 knodle.trainer.baseline.baseline INFO     Loss is: 0.24314838647842407
2020-11-23 15:58:28,337 knodle.trainer.baseline.baseline INFO     Epoch: 193
2020-11-23 15:58:28,345 knodle.trainer.baseline.baseline INFO     Loss is: 0.2426263839006424
2020-11-23 15:58:28,355 knodle.trainer.baseline.baseline INFO     Epoch: 194
2020-11-23 15:58:28,362 knodle.trainer.baseline.baseline INFO    

...

2020-11-23 15:58:38,449 knodle.trainer.baseline.baseline INFO     Epoch: 765
2020-11-23 15:58:38,457 knodle.trainer.baseline.baseline INFO     Loss is: 0.1370910108089447
2020-11-23 15:58:38,466 knodle.trainer.baseline.baseline INFO     Epoch: 766
2020-11-23 15:58:38,474 knodle.trainer.baseline.baseline INFO     Loss is: 0.13702911138534546
2020-11-23 15:58:38,483 knodle.trainer.baseline.baseline INFO     Epoch: 767
2020-11-23 15:58:38,491 knodle.trainer.baseline.baseline INFO     Loss is: 0.13696737587451935
2020-11-23 15:58:38,501 knodle.trainer.baseline.baseline INFO     Epoch: 768
2020-11-23 15:58:38,509 knodle.trainer.baseline.baseline INFO     Loss is: 0.13690577447414398
2020-11-23 15:58:38,518 knodle.trainer.baseline.baseline INFO     Epoch: 769
2020-11-23 15:58:38,526 knodle.trainer.baseline.baseline INFO     Loss is: 0.13684432208538055
2020-11-23 15:58:38,535 knodle.trainer.baseline.baseline INFO     Epoch: 770
2020-11-23 15:58:38,543 knodle.trainer.baseline.baseline INFO   

2020-11-23 15:58:39,291 knodle.trainer.baseline.baseline INFO     Epoch: 813
2020-11-23 15:58:39,299 knodle.trainer.baseline.baseline INFO     Loss is: 0.13428263366222382
2020-11-23 15:58:39,308 knodle.trainer.baseline.baseline INFO     Epoch: 814
2020-11-23 15:58:39,317 knodle.trainer.baseline.baseline INFO     Loss is: 0.1342274695634842
2020-11-23 15:58:39,325 knodle.trainer.baseline.baseline INFO     Epoch: 815
2020-11-23 15:58:39,334 knodle.trainer.baseline.baseline INFO     Loss is: 0.1341724395751953
2020-11-23 15:58:39,343 knodle.trainer.baseline.baseline INFO     Epoch: 816
2020-11-23 15:58:39,352 knodle.trainer.baseline.baseline INFO     Loss is: 0.13411752879619598
2020-11-23 15:58:39,361 knodle.trainer.baseline.baseline INFO     Epoch: 817
2020-11-23 15:58:39,370 knodle.trainer.baseline.baseline INFO     Loss is: 0.1340627521276474
2020-11-23 15:58:39,379 knodle.trainer.baseline.baseline INFO     Epoch: 818
2020-11-23 15:58:39,388 knodle.trainer.baseline.baseline INFO     

2020-11-23 15:58:40,181 knodle.trainer.baseline.baseline INFO     Epoch: 861
2020-11-23 15:58:40,188 knodle.trainer.baseline.baseline INFO     Loss is: 0.13177350163459778
2020-11-23 15:58:40,198 knodle.trainer.baseline.baseline INFO     Epoch: 862
2020-11-23 15:58:40,206 knodle.trainer.baseline.baseline INFO     Loss is: 0.13172408938407898
2020-11-23 15:58:40,215 knodle.trainer.baseline.baseline INFO     Epoch: 863
2020-11-23 15:58:40,223 knodle.trainer.baseline.baseline INFO     Loss is: 0.13167478144168854
2020-11-23 15:58:40,232 knodle.trainer.baseline.baseline INFO     Epoch: 864
2020-11-23 15:58:40,240 knodle.trainer.baseline.baseline INFO     Loss is: 0.13162559270858765
2020-11-23 15:58:40,250 knodle.trainer.baseline.baseline INFO     Epoch: 865
2020-11-23 15:58:40,259 knodle.trainer.baseline.baseline INFO     Loss is: 0.13157649338245392
2020-11-23 15:58:40,270 knodle.trainer.baseline.baseline INFO     Epoch: 866
2020-11-23 15:58:40,280 knodle.trainer.baseline.baseline INFO  

2020-11-23 15:58:41,023 knodle.trainer.baseline.baseline INFO     Epoch: 909
2020-11-23 15:58:41,031 knodle.trainer.baseline.baseline INFO     Loss is: 0.129520446062088
2020-11-23 15:58:41,041 knodle.trainer.baseline.baseline INFO     Epoch: 910
2020-11-23 15:58:41,049 knodle.trainer.baseline.baseline INFO     Loss is: 0.12947595119476318
2020-11-23 15:58:41,059 knodle.trainer.baseline.baseline INFO     Epoch: 911
2020-11-23 15:58:41,067 knodle.trainer.baseline.baseline INFO     Loss is: 0.1294315606355667
2020-11-23 15:58:41,077 knodle.trainer.baseline.baseline INFO     Epoch: 912
2020-11-23 15:58:41,085 knodle.trainer.baseline.baseline INFO     Loss is: 0.1293872594833374
2020-11-23 15:58:41,095 knodle.trainer.baseline.baseline INFO     Epoch: 913
2020-11-23 15:58:41,103 knodle.trainer.baseline.baseline INFO     Loss is: 0.12934309244155884
2020-11-23 15:58:41,112 knodle.trainer.baseline.baseline INFO     Epoch: 914
2020-11-23 15:58:41,120 knodle.trainer.baseline.baseline INFO     L

2020-11-23 15:58:41,878 knodle.trainer.baseline.baseline INFO     Epoch: 957
2020-11-23 15:58:41,886 knodle.trainer.baseline.baseline INFO     Loss is: 0.1274879425764084
2020-11-23 15:58:41,895 knodle.trainer.baseline.baseline INFO     Epoch: 958
2020-11-23 15:58:41,903 knodle.trainer.baseline.baseline INFO     Loss is: 0.1274477243423462
2020-11-23 15:58:41,912 knodle.trainer.baseline.baseline INFO     Epoch: 959
2020-11-23 15:58:41,921 knodle.trainer.baseline.baseline INFO     Loss is: 0.12740756571292877
2020-11-23 15:58:41,930 knodle.trainer.baseline.baseline INFO     Epoch: 960
2020-11-23 15:58:41,938 knodle.trainer.baseline.baseline INFO     Loss is: 0.1273675113916397
2020-11-23 15:58:41,947 knodle.trainer.baseline.baseline INFO     Epoch: 961
2020-11-23 15:58:41,955 knodle.trainer.baseline.baseline INFO     Loss is: 0.12732753157615662
2020-11-23 15:58:41,964 knodle.trainer.baseline.baseline INFO     Epoch: 962
2020-11-23 15:58:41,972 knodle.trainer.baseline.baseline INFO     

In [25]:
test(trainer.model, test_features=test_tensor, test_labels=test_label_tensor)

0.48241207


## kNN Model

In [26]:

model = LogisticRegressionModel(train_tensor.shape[1], OUTPUT_CLASSES)

trainer_config = TrainerConfig(model, criterion=cross_entropy_with_probs, optimizer_=SGD(model.parameters(), lr=5))

trainer = KnnDenoising(model, trainer_config=trainer_config)
trainer.train(inputs=train_tensor, rule_matches=train_rule_matches, epochs=1000, tfidf_values=train_tfidf, k=4)


2020-11-23 15:58:47,407 knodle.trainer.ds_model_trainer.ds_model_trainer INFO     Initalized trainer with custom model config: {'criterion': <function cross_entropy_with_probs at 0x13dfd51e0>, 'batch_size': 32, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 5
    momentum: 0
    nesterov: False
    weight_decay: 0
), 'output_classes': 2}
2020-11-23 15:58:47,408 knodle.trainer.knn_denoising.knn_denoising INFO     Start denoising labeling functions with k: 4.
2020-11-23 15:58:47,409 knodle.trainer.knn_denoising.knn_denoising INFO     This can take a while ...
800it [00:00, 47579.42it/s]
2020-11-23 15:58:47,576 knodle.trainer.knn_denoising.knn_denoising INFO     ======================================
2020-11-23 15:58:47,577 knodle.trainer.knn_denoising.knn_denoising INFO     Training starts
2020-11-23 15:58:47,578 knodle.trainer.knn_denoising.knn_denoising INFO     ======================================
2020-11-23 15:58:47,578 knodle.trainer.knn_denoising.knn_denoising INFO

2020-11-23 15:58:48,258 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.29030880331993103
2020-11-23 15:58:48,268 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 39
2020-11-23 15:58:48,276 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2893182933330536
2020-11-23 15:58:48,285 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 40
2020-11-23 15:58:48,293 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2883322238922119
2020-11-23 15:58:48,303 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 41
2020-11-23 15:58:48,310 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2873505651950836
2020-11-23 15:58:48,320 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 42
2020-11-23 15:58:48,327 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2863733768463135
2020-11-23 15:58:48,337 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 43
2020-11-23 15:58:48,344 knodle.trainer.knn_denois

2020-11-23 15:58:49,023 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 82
2020-11-23 15:58:49,031 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2508751451969147
2020-11-23 15:58:49,040 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 83
2020-11-23 15:58:49,049 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.25007525086402893
2020-11-23 15:58:49,058 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 84
2020-11-23 15:58:49,066 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.24927949905395508
2020-11-23 15:58:49,075 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 85
2020-11-23 15:58:49,084 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.24848787486553192
2020-11-23 15:58:49,093 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 86
2020-11-23 15:58:49,102 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2477003037929535
2020-11-23 15:58:49,111 knodle.trainer.knn_deno

2020-11-23 15:58:49,785 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 125
2020-11-23 15:58:49,793 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.22002635896205902
2020-11-23 15:58:49,802 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 126
2020-11-23 15:58:49,810 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21939031779766083
2020-11-23 15:58:49,820 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 127
2020-11-23 15:58:49,827 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2187577486038208
2020-11-23 15:58:49,837 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 128
2020-11-23 15:58:49,844 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21812854707241058
2020-11-23 15:58:49,854 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 129
2020-11-23 15:58:49,862 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21750280261039734
2020-11-23 15:58:49,872 knodle.trainer.kn

2020-11-23 15:58:50,540 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 168
2020-11-23 15:58:50,548 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.1955696940422058
2020-11-23 15:58:50,557 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 169
2020-11-23 15:58:50,565 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19506590068340302
2020-11-23 15:58:50,575 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 170
2020-11-23 15:58:50,583 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.1945648342370987
2020-11-23 15:58:50,592 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 171
2020-11-23 15:58:50,601 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19406643509864807
2020-11-23 15:58:50,610 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 172
2020-11-23 15:58:50,619 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19357070326805115
2020-11-23 15:58:50,628 knodle.trainer.knn

...

2020-11-23 15:59:01,908 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 813
2020-11-23 15:59:01,917 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08696243166923523
2020-11-23 15:59:01,926 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 814
2020-11-23 15:59:01,935 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08691266179084778
2020-11-23 15:59:01,945 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 815
2020-11-23 15:59:01,953 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08686298131942749
2020-11-23 15:59:01,963 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 816
2020-11-23 15:59:01,971 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08681347221136093
2020-11-23 15:59:01,981 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 817
2020-11-23 15:59:01,989 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08676405251026154
2020-11-23 15:59:01,998 knodle.trainer.k

2020-11-23 15:59:02,670 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 856
2020-11-23 15:59:02,678 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08492476493120193
2020-11-23 15:59:02,687 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 857
2020-11-23 15:59:02,695 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08487976342439651
2020-11-23 15:59:02,705 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 858
2020-11-23 15:59:02,712 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08483485877513885
2020-11-23 15:59:02,722 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 859
2020-11-23 15:59:02,730 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08479006588459015
2020-11-23 15:59:02,740 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 860
2020-11-23 15:59:02,748 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.0847453698515892
2020-11-23 15:59:02,758 knodle.trainer.kn

2020-11-23 15:59:03,428 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 899
2020-11-23 15:59:03,438 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08307904005050659
2020-11-23 15:59:03,447 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 900
2020-11-23 15:59:03,456 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08303819596767426
2020-11-23 15:59:03,465 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 901
2020-11-23 15:59:03,474 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08299744874238968
2020-11-23 15:59:03,483 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 902
2020-11-23 15:59:03,491 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08295678347349167
2020-11-23 15:59:03,500 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 903
2020-11-23 15:59:03,508 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08291621506214142
2020-11-23 15:59:03,517 knodle.trainer.k

2020-11-23 15:59:04,193 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 942
2020-11-23 15:59:04,202 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08140098303556442
2020-11-23 15:59:04,211 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 943
2020-11-23 15:59:04,220 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08136378228664398
2020-11-23 15:59:04,229 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 944
2020-11-23 15:59:04,238 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08132665604352951
2020-11-23 15:59:04,248 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 945
2020-11-23 15:59:04,256 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.0812896192073822
2020-11-23 15:59:04,265 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 946
2020-11-23 15:59:04,273 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08125264942646027
2020-11-23 15:59:04,283 knodle.trainer.kn

2020-11-23 15:59:04,958 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 985
2020-11-23 15:59:04,967 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07987004518508911
2020-11-23 15:59:04,976 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 986
2020-11-23 15:59:04,984 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07983603328466415
2020-11-23 15:59:04,993 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 987
2020-11-23 15:59:05,002 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07980211079120636
2020-11-23 15:59:05,011 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 988
2020-11-23 15:59:05,020 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07976825535297394
2020-11-23 15:59:05,029 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 989
2020-11-23 15:59:05,039 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07973446696996689
2020-11-23 15:59:05,048 knodle.trainer.k

In [27]:
test(trainer.model, test_features=test_tensor, test_labels=test_label_tensor)

0.4321608
